In [1]:
#Importing of relavent functions, pre-trained models, and libraries
import string
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    text_without_punctuation = text.translate(translator)
    return text_without_punctuation

def l1_distance(vectors):
    x, y = vectors
    return tf.reduce_sum(tf.abs(x - y), axis=1, keepdims=True)

import gensim.downloader
import numpy as np
from gensim.models import Word2Vec
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.losses import cosine_similarity
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Lambda, Dense
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

Embedder = gensim.downloader.load('word2vec-google-news-300')

In [2]:
#Reading in the training and test sets
Master = pd.read_csv("train.tsv",sep="\t")
Train,Dev = train_test_split(Master,test_size=0.1,random_state=42)
Test = pd.read_csv("dev.tsv",sep="\t")

In [3]:
#Pre-processing of training set

Train = Train.dropna()
Train = Train[~Train.apply(lambda row: row.str.strip().str.len().eq(0).any(), axis=1)]

for i in range(len(Train["question1"])):
    Train["question1"].iloc[i] = remove_punctuation(Train["question1"].iloc[i].lower())
    Train["question2"].iloc[i] = remove_punctuation(Train["question2"].iloc[i].lower())

Train = Train.dropna()
Train = Train[~Train.apply(lambda row: row.str.strip().str.len().eq(0).any(), axis=1)]

Q1 = Train["question1"].copy()
Q2 = Train["question2"].copy()

#Tokenization of training set
Q1_Tokens = Q1.copy()
Q2_Tokens = Q2.copy()
for i in range(len(Q1_Tokens)):
    Q1_Tokens.iloc[i] = Q1_Tokens.iloc[i].split()
    Q2_Tokens.iloc[i] = Q2_Tokens.iloc[i].split()

#Embedding of training set
Q1_Embedded = Q1_Tokens.copy()
Q2_Embedded = Q2_Tokens.copy()
for i in range(len(Q1_Embedded)):
  Sum_Q1 = np.zeros(300)
  Counter_Q1 = 0
  for word in Q1_Embedded.iloc[i]:
    if word in Embedder:
      Sum_Q1 = Sum_Q1 + Embedder[word]
      Counter_Q1 = Counter_Q1 + 1
  Q1_Embedded.iloc[i] = Sum_Q1 
  Sum_Q2 = np.zeros(300)
  Counter_Q2 = 0
  for word in Q2_Embedded.iloc[i]:
    if word in Embedder:
      Sum_Q2 = Sum_Q2 + Embedder[word]
      Counter_Q2 = Counter_Q2 + 1
  Q2_Embedded.iloc[i] = Sum_Q2


#Formatting inputs to match model's input format
Q1_Inputs = []
for i in range(len(Q1_Embedded)):
    #print(i)
    Q1_Inputs.append(Q1_Embedded.iloc[i].tolist())
Q1_Inputs = np.array(Q1_Inputs)

Q2_Inputs = []
for i in range(len(Q2_Embedded)):
    Q2_Inputs.append(Q2_Embedded.iloc[i].tolist())
Q2_Inputs = np.array(Q2_Inputs)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [4]:
#Pre-Processing of validation set
Dev = Dev.dropna()
Dev = Dev[~Dev.apply(lambda row: row.str.strip().str.len().eq(0).any(), axis=1)]

for i in range(len(Dev["question1"])):
    Dev["question1"].iloc[i] = remove_punctuation(Dev["question1"].iloc[i].lower())
    Dev["question2"].iloc[i] = remove_punctuation(Dev["question2"].iloc[i].lower())

Dev = Dev.dropna()
Dev = Dev[~Dev.apply(lambda row: row.str.strip().str.len().eq(0).any(), axis=1)]

Q1_Dev = Dev["question1"].copy()
Q2_Dev = Dev["question2"].copy()

#Tokenization of validation set
Q1_Dev_Tokens = Q1_Dev.copy()
Q2_Dev_Tokens = Q2_Dev.copy()
for i in range(len(Q1_Dev_Tokens)):
    Q1_Dev_Tokens.iloc[i] = Q1_Dev_Tokens.iloc[i].split()
    Q2_Dev_Tokens.iloc[i] = Q2_Dev_Tokens.iloc[i].split()

#Embedding of validation set
from gensim.models import Word2Vec
Q1_Embedded_Dev = Q1_Dev_Tokens.copy()
Q2_Embedded_Dev = Q2_Dev_Tokens.copy()

for i in range(len(Q1_Embedded_Dev)):
  Sum_Q1 = np.zeros(300)
  for word in Q1_Embedded_Dev.iloc[i]:
    if word in Embedder:
      Sum_Q1 = Sum_Q1 + (Embedder[word])
  Q1_Embedded_Dev.iloc[i] = Sum_Q1 
  Sum_Q2 = np.zeros(300)
  for word in Q2_Embedded_Dev.iloc[i]:
    if word in Embedder:
      Sum_Q2 = Sum_Q2 + (Embedder[word])
  Q2_Embedded_Dev.iloc[i] = Sum_Q2

#Formating embedding to match model's input format

Q1_Inputs_Dev = []
for i in range(len(Q1_Embedded_Dev)):
    Q1_Inputs_Dev.append(Q1_Embedded_Dev.iloc[i].tolist())
Q1_Inputs_Dev = np.array(Q1_Inputs_Dev)

Q2_Inputs_Dev = []
for i in range(len(Q2_Embedded_Dev)):
    Q2_Inputs_Dev.append(Q2_Embedded_Dev.iloc[i].tolist())
Q2_Inputs_Dev = np.array(Q2_Inputs_Dev)

In [5]:
#Pre-processing the test set
Test = Test.dropna()
Test = Test[~Test.apply(lambda row: row.str.strip().str.len().eq(0).any(), axis=1)]

for i in range(len(Test["question1"])):
    Test["question1"].iloc[i] = remove_punctuation(Test["question1"].iloc[i].lower())
    Test["question2"].iloc[i] = remove_punctuation(Test["question2"].iloc[i].lower())

Test = Test.dropna()
Test = Test[~Test.apply(lambda row: row.str.strip().str.len().eq(0).any(), axis=1)]

#Tokenization of the test set
Q1_Test = Test["question1"].copy()
Q2_Test = Test["question2"].copy()

Q1_Test_Tokens = Q1_Test.copy()
Q2_Test_Tokens = Q2_Test.copy()
for i in range(len(Q1_Test_Tokens)):
    Q1_Test_Tokens.iloc[i] = Q1_Test_Tokens.iloc[i].split()
    Q2_Test_Tokens.iloc[i] = Q2_Test_Tokens.iloc[i].split()

#Creating embeddings for the test set
from gensim.models import Word2Vec
Q1_Embedded_Test = Q1_Test_Tokens.copy()
Q2_Embedded_Test = Q2_Test_Tokens.copy()

for i in range(len(Q1_Embedded_Test)):
  Sum_Q1 = np.zeros(300)
  for word in Q1_Embedded_Test.iloc[i]:
    if word in Embedder:
      Sum_Q1 = Sum_Q1 + (Embedder[word])
  Q1_Embedded_Test.iloc[i] = Sum_Q1 
  Sum_Q2 = np.zeros(300)
  for word in Q2_Embedded_Test.iloc[i]:
    if word in Embedder:
      Sum_Q2 = Sum_Q2 + (Embedder[word])
  Q2_Embedded_Test.iloc[i] = Sum_Q2 

#Formatting the inputs to match the inputs of the model
Q1_Inputs_Test = []
for i in range(len(Q1_Embedded_Test)):
    Q1_Inputs_Test.append(Q1_Embedded_Test.iloc[i].tolist())
Q1_Inputs_Test = np.array(Q1_Inputs_Test)

Q2_Inputs_Test = []
for i in range(len(Q2_Embedded_Test)):
    Q2_Inputs_Test.append(Q2_Embedded_Test.iloc[i].tolist())
Q2_Inputs_Test = np.array(Q2_Inputs_Test)

In [6]:
#Clear up ram as we don't need the embedder anymore
Embedder = 0

In [7]:
#Hyperparameter Tuning
from sklearn.model_selection import ParameterGrid
embedding_dim = 300
threshold = 0.5
results = []

# Define the hyperparameter grid
param_grid = {
    'learning_rate': [0.001, 0.0001],
    'hidden_layers': [2, 3, 4],
    'units_per_layer': [128, 256, 512],
    'batch_size': [32, 64],
    'epochs': [10, 15]
}


# Create a list of all possible hyperparameter combinations
param_combinations = list(ParameterGrid(param_grid))

# Iterate through each combination
for params in param_combinations:
    print(f"Training model with hyperparameters: {params}")

    # Reinitialize the Siamese network with new hyperparameters
    shared_network = keras.Sequential([
        layers.Dense(embedding_dim, activation='relu', input_shape=(embedding_dim,)),
        *[layers.Dense(params['units_per_layer'], activation='relu') for _ in range(params['hidden_layers'])],
    ])

    left_input = layers.Input(shape=(embedding_dim,))
    right_input = layers.Input(shape=(embedding_dim,))

    encoded_left = shared_network(left_input)
    encoded_right = shared_network(right_input)

    distance = Lambda(l1_distance)([encoded_left, encoded_right])
    similarity_prediction = Dense(1, activation='sigmoid')(distance)

    siamese_model = keras.Model(inputs=[left_input, right_input], outputs=similarity_prediction)

    custom_optimizer = Adam(learning_rate=params['learning_rate'])
    siamese_model.compile(loss='binary_crossentropy', optimizer=custom_optimizer, metrics=['accuracy'])

    # Training the model
    siamese_model.fit(
        [Q1_Inputs, Q2_Inputs],
        np.array(Train["is_duplicate"]),
        batch_size=params['batch_size'],
        epochs=params['epochs'],  # Use the specified number of epochs
    )


    # Evaluating on the validation set
    dev_predictions = siamese_model.predict([Q1_Inputs_Dev, Q2_Inputs_Dev])
    dev_binary_predictions = (dev_predictions > threshold).astype(int)
    dev_accuracy = accuracy_score(Dev["is_duplicate"].tolist(), dev_binary_predictions)

    result = {'hyperparameters': params, 'validation_accuracy': dev_accuracy}
    results.append(result)
    
    print(f"Validation Accuracy: {dev_accuracy * 100:.2f}%\n")
    tf.keras.backend.clear_session()



Training model with hyperparameters: {'batch_size': 32, 'epochs': 10, 'hidden_layers': 2, 'learning_rate': 0.001, 'units_per_layer': 128}
Epoch 1/10
10233/10233 [==============================] - 12s 1ms/step - loss: 0.5148 - accuracy: 0.7355
Epoch 2/10
10233/10233 [==============================] - 12s 1ms/step - loss: 0.4673 - accuracy: 0.7727
Epoch 3/10
10233/10233 [==============================] - 12s 1ms/step - loss: 0.4454 - accuracy: 0.7878
Epoch 4/10
10233/10233 [==============================] - 12s 1ms/step - loss: 0.4292 - accuracy: 0.7985
Epoch 5/10
10233/10233 [==============================] - 12s 1ms/step - loss: 0.4153 - accuracy: 0.8077
Epoch 6/10
10233/10233 [==============================] - 13s 1ms/step - loss: 0.4023 - accuracy: 0.8163
Epoch 7/10
10233/10233 [==============================] - 12s 1ms/step - loss: 0.3906 - accuracy: 0.8233
Epoch 8/10
10233/10233 [==============================] - 13s 1ms/step - loss: 0.3795 - accuracy: 0.8298
Epoch 9/10
10233/10233

Epoch 1/10
10233/10233 [==============================] - 22s 2ms/step - loss: 0.6561 - accuracy: 0.6415
Epoch 2/10
10233/10233 [==============================] - 24s 2ms/step - loss: 0.6133 - accuracy: 0.6830
Epoch 3/10
10233/10233 [==============================] - 22s 2ms/step - loss: 0.5900 - accuracy: 0.7030
Epoch 4/10
10233/10233 [==============================] - 21s 2ms/step - loss: 0.5720 - accuracy: 0.7176
Epoch 5/10
10233/10233 [==============================] - 22s 2ms/step - loss: 0.5578 - accuracy: 0.7290
Epoch 6/10
10233/10233 [==============================] - 22s 2ms/step - loss: 0.5441 - accuracy: 0.7396
Epoch 7/10
10233/10233 [==============================] - 21s 2ms/step - loss: 0.5316 - accuracy: 0.7487
Epoch 8/10
10233/10233 [==============================] - 22s 2ms/step - loss: 0.5213 - accuracy: 0.7558
Epoch 9/10
10233/10233 [==============================] - 19s 2ms/step - loss: 0.5111 - accuracy: 0.7628
Epoch 10/10
1137/1137 [==============================] 

Validation Accuracy: 75.03%

Training model with hyperparameters: {'batch_size': 32, 'epochs': 10, 'hidden_layers': 2, 'learning_rate': 0.001, 'units_per_layer': 512}
Epoch 1/10
10233/10233 [==============================] - 33s 3ms/step - loss: 0.5335 - accuracy: 0.7227
Epoch 2/10
10233/10233 [==============================] - 33s 3ms/step - loss: 0.4773 - accuracy: 0.7659
Epoch 3/10
10233/10233 [==============================] - 30s 3ms/step - loss: 0.4552 - accuracy: 0.7811
Epoch 4/10
10233/10233 [==============================] - 32s 3ms/step - loss: 0.4381 - accuracy: 0.7922
Epoch 5/10
10233/10233 [==============================] - 33s 3ms/step - loss: 0.4244 - accuracy: 0.8015
Epoch 6/10
10233/10233 [==============================] - 32s 3ms/step - loss: 0.4119 - accuracy: 0.8096
Epoch 7/10
10233/10233 [==============================] - 33s 3ms/step - loss: 0.3992 - accuracy: 0.8180
Epoch 8/10
10233/10233 [==============================] - 32s 3ms/step - loss: 0.3875 - accuracy: 

Validation Accuracy: 81.26%

Training model with hyperparameters: {'batch_size': 32, 'epochs': 10, 'hidden_layers': 2, 'learning_rate': 0.0001, 'units_per_layer': 128}
Epoch 1/10
10233/10233 [==============================] - 12s 1ms/step - loss: 0.7126 - accuracy: 0.6437
Epoch 2/10
10233/10233 [==============================] - 12s 1ms/step - loss: 0.5981 - accuracy: 0.6995
Epoch 3/10
10233/10233 [==============================] - 12s 1ms/step - loss: 0.5731 - accuracy: 0.7184
Epoch 4/10
10233/10233 [==============================] - 12s 1ms/step - loss: 0.5557 - accuracy: 0.7318
Epoch 5/10
10233/10233 [==============================] - 12s 1ms/step - loss: 0.5414 - accuracy: 0.7420
Epoch 6/10
10233/10233 [==============================] - 12s 1ms/step - loss: 0.5273 - accuracy: 0.7520
Epoch 7/10
10233/10233 [==============================] - 12s 1ms/step - loss: 0.5145 - accuracy: 0.7613
Epoch 8/10
10233/10233 [==============================] - 12s 1ms/step - loss: 0.5025 - accuracy:

Epoch 1/10
10233/10233 [==============================] - 18s 2ms/step - loss: 0.5864 - accuracy: 0.6925
Epoch 2/10
10233/10233 [==============================] - 19s 2ms/step - loss: 0.4966 - accuracy: 0.7524
Epoch 3/10
10233/10233 [==============================] - 19s 2ms/step - loss: 0.4628 - accuracy: 0.7760
Epoch 4/10
10233/10233 [==============================] - 19s 2ms/step - loss: 0.4376 - accuracy: 0.7921
Epoch 5/10
10233/10233 [==============================] - 19s 2ms/step - loss: 0.4169 - accuracy: 0.8056
Epoch 6/10
10233/10233 [==============================] - 19s 2ms/step - loss: 0.3991 - accuracy: 0.8166
Epoch 7/10
10233/10233 [==============================] - 18s 2ms/step - loss: 0.3831 - accuracy: 0.8265
Epoch 8/10
10233/10233 [==============================] - 19s 2ms/step - loss: 0.3685 - accuracy: 0.8358
Epoch 9/10
10233/10233 [==============================] - 19s 2ms/step - loss: 0.3549 - accuracy: 0.8438
Epoch 10/10
1137/1137 [==============================] 

...

Epoch 1/15
5117/5117 [==============================] - 29s 6ms/step - loss: 0.6638 - accuracy: 0.6295
Epoch 2/15
5117/5117 [==============================] - 28s 5ms/step - loss: 0.6588 - accuracy: 0.6303
Epoch 3/15
5117/5117 [==============================] - 28s 6ms/step - loss: 0.6588 - accuracy: 0.6303
Epoch 4/15
5117/5117 [==============================] - 29s 6ms/step - loss: 0.6588 - accuracy: 0.6303
Epoch 5/15
5117/5117 [==============================] - 30s 6ms/step - loss: 0.6588 - accuracy: 0.6303
Epoch 6/15
5117/5117 [==============================] - 31s 6ms/step - loss: 0.6588 - accuracy: 0.6303
Epoch 7/15
5117/5117 [==============================] - 30s 6ms/step - loss: 0.6588 - accuracy: 0.6303
Epoch 8/15
5117/5117 [==============================] - 32s 6ms/step - loss: 0.6588 - accuracy: 0.6303
Epoch 9/15
5117/5117 [==============================] - 31s 6ms/step - loss: 0.6588 - accuracy: 0.6303
Epoch 10/15
5117/5117 [==============================] - 29s 6ms/step - l

Epoch 1/15
5117/5117 [==============================] - 13s 3ms/step - loss: 0.7057 - accuracy: 0.6210
Epoch 2/15
5117/5117 [==============================] - 13s 3ms/step - loss: 0.6116 - accuracy: 0.6969
Epoch 3/15
5117/5117 [==============================] - 13s 2ms/step - loss: 0.5777 - accuracy: 0.7183
Epoch 4/15
5117/5117 [==============================] - 12s 2ms/step - loss: 0.5552 - accuracy: 0.7332
Epoch 5/15
5117/5117 [==============================] - 13s 2ms/step - loss: 0.5376 - accuracy: 0.7460
Epoch 6/15
5117/5117 [==============================] - 13s 3ms/step - loss: 0.5225 - accuracy: 0.7568
Epoch 7/15
5117/5117 [==============================] - 13s 3ms/step - loss: 0.5096 - accuracy: 0.7657
Epoch 8/15
5117/5117 [==============================] - 13s 3ms/step - loss: 0.4968 - accuracy: 0.7751
Epoch 9/15
5117/5117 [==============================] - 13s 2ms/step - loss: 0.4844 - accuracy: 0.7832
Epoch 10/15
5117/5117 [==============================] - 13s 3ms/step - l

Epoch 1/15
5117/5117 [==============================] - 21s 4ms/step - loss: 0.6681 - accuracy: 0.6435
Epoch 2/15
5117/5117 [==============================] - 21s 4ms/step - loss: 0.6052 - accuracy: 0.7012
Epoch 3/15
5117/5117 [==============================] - 21s 4ms/step - loss: 0.5729 - accuracy: 0.7208
Epoch 4/15
5117/5117 [==============================] - 20s 4ms/step - loss: 0.5492 - accuracy: 0.7381
Epoch 5/15
5117/5117 [==============================] - 20s 4ms/step - loss: 0.5292 - accuracy: 0.7526
Epoch 6/15
5117/5117 [==============================] - 21s 4ms/step - loss: 0.5117 - accuracy: 0.7650
Epoch 7/15
5117/5117 [==============================] - 21s 4ms/step - loss: 0.4953 - accuracy: 0.7765
Epoch 8/15
5117/5117 [==============================] - 21s 4ms/step - loss: 0.4802 - accuracy: 0.7867
Epoch 9/15
5117/5117 [==============================] - 21s 4ms/step - loss: 0.4658 - accuracy: 0.7965
Epoch 10/15
5117/5117 [==============================] - 20s 4ms/step - l

Validation Accuracy: 76.66%

Training model with hyperparameters: {'batch_size': 64, 'epochs': 15, 'hidden_layers': 4, 'learning_rate': 0.0001, 'units_per_layer': 512}


Epoch 1/15
5117/5117 [==============================] - 30s 6ms/step - loss: 0.5577 - accuracy: 0.6898
Epoch 2/15
5117/5117 [==============================] - 29s 6ms/step - loss: 0.4932 - accuracy: 0.7524
Epoch 3/15
5117/5117 [==============================] - 29s 6ms/step - loss: 0.4542 - accuracy: 0.7803
Epoch 4/15
5117/5117 [==============================] - 31s 6ms/step - loss: 0.4211 - accuracy: 0.8014
Epoch 5/15
5117/5117 [==============================] - 32s 6ms/step - loss: 0.3920 - accuracy: 0.8201
Epoch 6/15
5117/5117 [==============================] - 29s 6ms/step - loss: 0.3638 - accuracy: 0.8366
Epoch 7/15
5117/5117 [==============================] - 29s 6ms/step - loss: 0.3383 - accuracy: 0.8523
Epoch 8/15
5117/5117 [==============================] - 29s 6ms/step - loss: 0.3143 - accuracy: 0.8671
Epoch 9/15
5117/5117 [==============================] - 31s 6ms/step - loss: 0.2920 - accuracy: 0.8802
Epoch 10/15
5117/5117 [==============================] - 29s 6ms/step - l

In [7]:
tf.keras.backend.clear_session()

#Initialization of the Siamese network with twin multi-layer perceptron models using the best model derived from grid search 
embedding_dim = 300

shared_network = keras.Sequential([
    layers.Dense(300, activation='relu', input_shape=(embedding_dim,)),
    layers.Dense(512, activation='relu'),
    layers.Dense(512, activation='relu'),
#    layers.Dense(512, activation='relu'),
#    layers.Dense(512, activation='relu'),
])

"""early_stopping = EarlyStopping(
    monitor='val_accuracy', 
    patience=5,           
    restore_best_weights=True  
)"""

left_input = layers.Input(shape=(embedding_dim,))
right_input = layers.Input(shape=(embedding_dim,))

encoded_left = shared_network(left_input)
encoded_right = shared_network(right_input)

distance = Lambda(l1_distance)([encoded_left, encoded_right])
similarity_prediction = Dense(1, activation='sigmoid')(distance)

siamese_model = keras.Model(inputs=[left_input, right_input], outputs=similarity_prediction)

custom_learning_rate = 0.0001
custom_optimizer = Adam(learning_rate=custom_learning_rate)

siamese_model.compile(loss='binary_crossentropy', optimizer=custom_optimizer, metrics=['accuracy'])

In [8]:
#Training of the model on the full data set

siamese_model.fit(
    [np.concatenate((Q1_Inputs, Q1_Inputs_Dev), axis=0), np.concatenate((Q2_Inputs, Q2_Inputs_Dev), axis=0)
],  # Your question embeddings
    np.concatenate([Train["is_duplicate"], Dev["is_duplicate"]], axis=0),  # Similarity labels (0 for dissimilar, 1 for similar)
    batch_size=32,
    epochs=15,
    #validation_split = 0.1
    #validation_data=([Q1_Inputs_Dev, Q2_Inputs_Dev], np.array(Dev["is_duplicate"]))
    #,callbacks=[early_stopping]
)

Epoch 1/15
11370/11370 [==============================] - 35s 3ms/step - loss: 0.5704 - accuracy: 0.6963
Epoch 2/15
11370/11370 [==============================] - 37s 3ms/step - loss: 0.4853 - accuracy: 0.7611
Epoch 3/15
11370/11370 [==============================] - 38s 3ms/step - loss: 0.4500 - accuracy: 0.7859
Epoch 4/15
11370/11370 [==============================] - 37s 3ms/step - loss: 0.4234 - accuracy: 0.8016
Epoch 5/15
11370/11370 [==============================] - 40s 4ms/step - loss: 0.4006 - accuracy: 0.8158
Epoch 6/15
11370/11370 [==============================] - 37s 3ms/step - loss: 0.3812 - accuracy: 0.8282
Epoch 7/15
11370/11370 [==============================] - 37s 3ms/step - loss: 0.3631 - accuracy: 0.8392
Epoch 8/15
11370/11370 [==============================] - 43s 4ms/step - loss: 0.3467 - accuracy: 0.8489
Epoch 9/15
11370/11370 [==============================] - 41s 4ms/step - loss: 0.3311 - accuracy: 0.8589
Epoch 10/15
11370/11370 [==============================

In [9]:
#Predicting the training set
train_predictions = siamese_model.predict([Q1_Inputs, Q2_Inputs])
threshold = 0.5
train_binary_predictions = (train_predictions > threshold).astype(int)
training_accuracy = accuracy_score(Train["is_duplicate"].tolist(), train_binary_predictions)
print(f"Training Accuracy: {training_accuracy * 100:.2f}%")


10233/10233 [==============================] - 10s 949us/step
Training Accuracy: 91.86%


In [17]:
#Predicting the test set
test_predictions = siamese_model.predict([Q1_Inputs_Test, Q2_Inputs_Test])
threshold = 0.5
test_binary_predictions = (test_predictions > threshold).astype(int)
#Calculating Test Accuracy
test_accuracy = accuracy_score(Test["is_duplicate"].tolist(), test_binary_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
#Calculating Test F1 Score
test_f1_score = f1_score(Test["is_duplicate"].tolist(), test_binary_predictions)
print(f"Test F1 Score: {test_f1_score * 100:.2f}%")


1264/1264 [==============================] - 1s 828us/step
Test Accuracy: 82.23%
Test F1 Score: 76.59%


In [16]:
#Exploratory Data Analysis on training points that we failed to predict correctly

Train_Incorrect = []

for i in range(len(Train["is_duplicate"])):
    if Train["is_duplicate"].iloc[i] != train_binary_predictions[i]:
        Train_Incorrect.append(i)

Train_Incorrect = Train.iloc[Train_Incorrect,]

print(Train_Incorrect)

            id    qid1    qid2  \
215014  297445  419811  357156   
159037  295046  417037  417038   
129104  341643  469510  469511   
311452  394357   97876  527242   
21037   173131  188364  267158   
...        ...     ...     ...   
82798   337217   23547  119932   
11534    37539   68294   68295   
341097   70696  121784  121785   
270936  288369  409316  397015   
194027   47789   23696   85293   

                                                question1  \
215014  how do i promote business blogs with social media   
159037                            why is 1700 a leap year   
129104  what are some examples of behavioral adaptions...   
311452             how did rahul dravid play for scotland   
21037      why do people who cant afford kids make babies   
...                                                   ...   
82798   extraterrestrial life what is the most undenia...   
11534   what is the meaning of orthogonality in signal...   
341097  what are the advantages of a sales

In [13]:
#Exploratory Data Analysis on test points that we failed to predict correctly

Test_Incorrect = []

for i in range(len(Test["is_duplicate"])):
    if Test["is_duplicate"].iloc[i] != test_binary_predictions[i]:
        Test_Incorrect.append(i)

Test_Incorrect = Test.iloc[Test_Incorrect,]

print(Test_Incorrect)

           id    qid1    qid2  \
5      323679  449673  311332   
6      144933   10927  229257   
7      198597  299859  299860   
8      382732  443182  514615   
19     308638  432498  432499   
...       ...     ...     ...   
40412  308976  432898  432899   
40420  144862  229155  229156   
40422   83147   76325  140832   
40423  146181  230973  230974   
40426  258093  373551  373552   

                                               question1  \
5      i am 25 year old guy and never had a girlfrien...   
6      what does a good answer on quora look like wha...   
7            what are the best things to do in hong kong   
8                  why is my life getting so complicated   
19     is it a bad time to buy a condo or a house in ...   
...                                                  ...   
40412  what are the major purpose of chhattisgarh vid...   
40420                       do women cheat more than men   
40422          how do i register a free minecraft server   
404

In [14]:
#pd.DataFrame(results).to_csv("/Users/hongruih/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/Year 3 Semester One/CS3244/CS3244 Project/Hyperparameter Tuning/results.csv")
pd.DataFrame(Train_Incorrect).to_csv("/Users/hongruih/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/Year 3 Semester One/CS3244/CS3244 Project/Hyperparameter Tuning/Train_Incorrect.csv")
pd.DataFrame(Test_Incorrect).to_csv("/Users/hongruih/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/Year 3 Semester One/CS3244/CS3244 Project/Hyperparameter Tuning/Test_Incorrect.csv")